## Test the usage

In [1]:
from wrapper import *
import ipdb
import pickle
import os
from tqdm import tqdm
import numpy as np

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
# load the NNLM model
tool = tools('pretrained', epoch=20)

[!] Load the model from ./ckpt/pretrained/epoch_12.pt, threshold (0, 20)


### SRF

In [3]:
def load_data_corpus(path):
    corpus = []
    with open(path) as f:
        for line in f.readlines():
            corpus.append(''.join(line.strip().split()))
    return corpus

In [4]:
dataset = 'xiaohuangji'
batch_size, filte = 128, 300
src = load_data_corpus(f'../GPT2-Chinese/data/{dataset}/src-train.txt')
tgt = load_data_corpus(f'../GPT2-Chinese/data/{dataset}/tgt-train.txt')

# filte the target sentences
tgtl = np.argsort([len(i) for i in tgt])
tgtp = [tgt[i] for i in tgtl][:filte]

tgtidx = [tgt.index(i) for i in tgtp]

In [ ]:
print(f'Data size: src({len(src)}), tgt({len(tgtp)})')

Data size: src(45000), tgt(300)


In [ ]:
r = np.zeros([len(src), len(tgtp)])
for idx, s in tqdm(enumerate(src)):
    idy = 0
    while idy < len(tgtp):
        tail = idy + batch_size
        tt = tgtp[idy:tail]
        batch_size = len(tt)
        ss = [s for i in range(batch_size)]
        r[idx, idy:tail] = tool.get_SRF(ss, tt)
        idy = tail
        
with open(f'./data/{dataset}/matrix.pkl', 'wb') as f:
    pickle.dump([tgtidx, r], f)
    print(f'[!] end of pickle the object: {r.shape}')

11it [00:11,  1.02it/s]

In [8]:
with open(f'./data/{dataset}/matrix.pkl', 'rb') as f:
    tidx, matrix = pickle.load(f)

[[0.47248876 1.94709467 0.74942765 2.5194506  0.39764487 0.57951906
  0.7668182  3.17883583 2.13465986 3.44263773]
 [0.52768441 2.25581139 1.06862997 2.42722384 0.69509303 0.67380402
  0.9950032  4.12490166 2.6424781  4.42226059]
 [0.39974221 1.52474906 0.74331209 1.88420056 0.3807821  0.53631289
  0.81957621 2.36085011 1.45164923 2.98265964]
 [0.66064972 3.87541943 1.26338225 3.69883117 0.82828571 0.96867288
  1.32011143 5.08266478 3.17496295 5.1427725 ]
 [1.05380553 3.90298561 1.64043447 4.43709175 0.97332509 0.88917991
  1.38573616 7.25417456 3.96840404 7.3214302 ]
 [1.05822643 5.00268418 1.16619344 5.11389147 0.91559071 1.06078936
  1.46530855 6.86534499 4.69142852 6.98671371]
 [0.86953262 4.28920069 1.09273181 4.84397897 0.89962789 0.76202829
  1.25106534 6.64960444 3.95701851 6.70165972]
 [0.49579413 2.41718955 1.06602517 2.65581524 0.61773401 0.67624124
  1.05259331 3.3878668  2.60798014 3.82112264]
 [0.38701818 2.17404513 1.09727439 2.29420905 0.41989987 0.59883053
  0.8169007 